<a href="https://colab.research.google.com/github/wodnjse/big-data-analysis/blob/main/Web-Crawling/coffeebeankorea.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
!apt-get update
!apt install chromium-chromedriver

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency t

In [25]:
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


In [26]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys
import time

In [27]:
# 코랩이기 때문에 해주는 코드
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)

In [28]:
# 커피빈 매장 페이지 열기

url = 'https://www.coffeebeankorea.com/store/store.asp'
driver.get(url)

In [29]:
# 매장 정보 페이지를 열기 위해 자바스크립트 함수인 storePop2를 호출
driver.execute_script('storePop2(1)')

In [30]:
# page_source
html = driver.page_source

In [31]:
soupCB1 = bs(html, 'html.parser')

In [ ]:
print(soupCB1.prettify())

In [33]:
# 지점 이름 가져오기
store_name_h2 = soupCB1.select('div.store_txt > h2')
store_name_h2

[<h2>학동역 DT점</h2>]

In [34]:
store_name = store_name_h2[0].string
store_name

'학동역 DT점'

In [36]:
# 매장정보
store_info = soupCB1.select('div.store_txt > table.store_table > tbody > tr > td')
store_info

[<td>평일 06:30~22:00ㅣ주말/공휴일 07:30~22:00</td>,
 <td>DT(드라이브 스루) 매장입니다. (주차는 불가)</td>,
 <td>서울시 강남구 학동로 211 1층  <!--span class="lot">(서울시 강남구 학동로 211 1층)</span--></td>,
 <td>02-3444-9973</td>]

In [44]:
# 주소 가져오기
store_address_info = list(store_info[2])
store_address_info

['서울시 강남구 학동로 211 1층  ', 'span class="lot">(서울시 강남구 학동로 211 1층)</span']

In [45]:
store_address  = store_address_info[0]
store_address

'서울시 강남구 학동로 211 1층  '

In [46]:
# 전화번호 가져오기
store_number = store_info[3].string
store_number

'02-3444-9973'

In [47]:
# 전체 매장 정보 크롤링 하기
# 매장 한 개에 대한 크롤링 작업을 매장의 갯수만큼 반복처리하면서 result 리스트에 저장한다.
# 완성된 result 리스트를 csv 파일에 저장하는 코드를 작성

In [42]:
import pandas as pd
import datetime

In [53]:
def Bean_shop(result):
  url = 'https://www.coffeebeankorea.com/store/store.asp'

  options = webdriver.ChromeOptions()
  options.add_argument('--headless')  # Head-less 설정
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver', options=options)

  for i in range(1, 400):
    driver.get(url)
    time.sleep(1)  # 웹 페이지에 연결할 동안 1초 대기

    try:
      driver.execute_script('storePop2(%d)' %i)
      time.sleep(1)  # 스크립트 실행할 동안 1초 대기
      html = driver.page_source
      soupCB = bs(html, 'html.parser')
      
      store_name_h2 = soupCB.select('div.store_txt > h2')
      store_name = store_name_h2[0].string
      print(store_name)  # 매장 이름 출력

      store_info = soupCB1.select('div.store_txt > table.store_table > tbody > tr > td')
      store_address_info = list(store_info[2])
      store_address  = store_address_info[0]

      store_number = store_info[3].string

      result.append([store_name] + [store_address] + [store_number])

    except:
      continue

  return

result = []

print('CoffeeBean store crawling >>>>>>>>>>>>>>>>>>>>>>>>>>>')
Bean_shop(result)

df = pd.DataFrame(result, columns = ('name', 'address', 'number'))
df.to_csv('/content/drive/MyDrive/Colab Notebooks/빅데이터분석2/CoffeeBean.csv', encoding = 'cp949')

CoffeeBean store crawling >>>>>>>>>>>>>>>>>>>>>>>>>>>
학동역 DT점
차병원점
강남대로점
강남에스점
청담에스점
신사점
압구정역점
역삼점
양재스포타임점
청담성당점
영동점
도곡점
영동고앞점
압구정시티점
압구정로데오점
서초우성점
논현팍스타워점
삼성오크우드점
트레이드타워점
삼성봉은사거리점
반포엘루체점
잠실신천점
서초지파이브점
논현동수면센터점
삼성전자강남사옥점
삼성생명강남사옥점
방배카페골목점
역삼포스틸타워뒷점
테헤란로하이닉스뒷점
삼성봉은사로점
방이점
매봉역점
남부터미널역점
강남역랭기지타워점
도심공항타워점
청담카페점
광화문점
경희궁의아침점
홍대역점
순화점
종로관철동점
소공동삼화빌딩점
강변테크노마트점
외대앞점
고대안암역점
순화동라마다호텔점
세종문화회관옆점
신촌점
건대스타시티점
하나은행경복궁역점
중계동은행사거리점
서울스퀘어점
목동점
여의도34번지점
여의도교보증권점
시흥홈플러스점
여의도순복음교회옆점
부산광복동점
인천구월동점
대전탄방동점
일산라페스타점
일산웨스턴돔1호점
분당서현점
분당플라자점
분당차병원점
분당수내점
동탄라마다호텔점
창원상남동점
청주지웰시티점
무교CBTL점
국기원사거리점
무교점
남대문점
올림픽플라자점
대학로마로니에점
인사점
테헤란로비젼타워점
서울대입구역점
코엑스인터점
뉴압구정점
디큐브시티점
충무로극동빌딩점
현대자동차여의도점
포스코사거리점
선릉로93길점
강남역먹자골목점
삼성로점
선릉로86길점
안국역점
뉴교보타워옆점
IFC MALL2호점
상암SBS미디어센터점
마리오아울렛점
삼성금융플라자점
현대자동차남양연구소점
뉴가로수길점
서초뱅뱅사거리점
현대자동차성내점
수원인계점
판교아브뉴프랑점
일산원마운트점
남양연구소C1식당점
여의도교보증권카페점
압구정현대아파트점
역삼역4번출구점
시청역7번출구점
부평문화의거리점
강남대로센터점
구로G-ValleyMall
내방역7번출구유중아트점
도심공항점
타임스퀘어점
차병원사거리점
CBTL(커피빈)파르나스몰점
광화문역1번출구점
남양연구소PT동점
현대프리미엄아울렛김포점
여의도신영증권별관점
이태원제일기획옆점
강남역